In [ ]:
import pandas as pd
import numpy as np
import string
import re

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import bahasa baku dan Sarcastic comments

In [2]:
sentimen_public = pd.read_csv('../datasets/all_comments_temp.csv')
sentimen_public

,id,username,comment,num_of_likes,published_at
0,N3JZxBTC1Zw,@alvinvinoza,Naikin gajinya 🔥🔥,962,2025-03-12T14:42:39Z
1,9YFbtgNsSeA,@MuklasinMuklasin-k6w,"Kalo tidak ada hukuman mati, sampek kiamat pun...",847,2025-02-25T22:31:15Z
2,9YFbtgNsSeA,@infotodax,"Gua gak kebayang, Gaji elu berapa... Tapi kese...",683,2025-02-26T03:59:00Z
3,N3JZxBTC1Zw,@febbriyansaputra4132,Tolong host nya di support dan di lindungi ini,552,2025-03-19T13:20:14Z
4,N3JZxBTC1Zw,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...,535,2025-03-06T16:28:31Z
...,...,...,...,...,...
7549,https://x.com/KPK_RI/status/1704722123636539748,hedianti,@KPK_RI Halah..selama korupsi ga hukum mati......,5,Thu Sep 21 06:56:32 +0000 2023
7550,https://x.com/KPK_RI/status/1704722123636539748,Charles39953441,@KPK_RI beberapa Dirut pertamina jadi tersangk...,3,Thu Sep 21 07:16:03 +0000 2023
7551,https://x.com/KPK_RI/status/1704722123636539748,Aryabarcelona,@KPK_RI 10 tahun baru terungkap beneran gak si...,3,Thu Sep 21 07:23:35 +0000 2023
7552,https://x.com/KPK_RI/status/1704722123636539748,harley_doni,@KPK_RI Masak KA sebodoh itu ya. Apakah mmg me...,3,Thu Sep 21 07:59:09 +0000 2023


In [3]:
print(sentimen_public.info())
print('\nMissing Values: \n',sentimen_public.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7554 entries, 0 to 7553
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            7554 non-null   object
 1   username      7554 non-null   object
 2   comment       7554 non-null   object
 3   num_of_likes  7554 non-null   int64 
 4   published_at  7554 non-null   object
dtypes: int64(1), object(4)
memory usage: 295.2+ KB
None

Missing Values: 
 id              0
username        0
comment         0
num_of_likes    0
published_at    0
dtype: int64


In [8]:
sentimen_public = sentimen_public[['username', 'comment']]
sentimen_public

,username,comment
0,@alvinvinoza,naikin gajinya
1,@MuklasinMuklasin-k6w,kalo tidak ada hukuman mati sampek kiamat pun...
2,@infotodax,gua gak kebayang gaji elu berapa tapi kesera...
3,@febbriyansaputra4132,tolong host nya di support dan di lindungi ini
4,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...
...,...,...
7549,hedianti,halah selama korupsi ga hukum mati cmn beber...
7550,Charles39953441,beberapa dirut pertamina jadi tersangka setela...
7551,Aryabarcelona,tahun baru terungkap beneran gak si kerjanya
7552,harley_doni,masak ka sebodoh itu ya apakah mmg membodohka...


### convert corpus to lower case

In [9]:
sentimen_public['comment'] = sentimen_public['comment'].apply(lambda content: " ".join(words.lower() for words in content.split()))
sentimen_public['comment']

C:\Users\alber\AppData\Local\Temp\ipykernel_6952\2563371219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentimen_public['comment'] = sentimen_public['comment'].apply(lambda content: " ".join(words.lower() for words in content.split()))


0                                          naikin gajinya
1       kalo tidak ada hukuman mati sampek kiamat pun ...
2       gua gak kebayang gaji elu berapa tapi keseraka...
3          tolong host nya di support dan di lindungi ini
4       harusnya berita ya kayak begini dibawa santai ...
                              ...                        
7549    halah selama korupsi ga hukum mati cmn beberap...
7550    beberapa dirut pertamina jadi tersangka setela...
7551         tahun baru terungkap beneran gak si kerjanya
7552    masak ka sebodoh itu ya apakah mmg membodohkan...
7553    nah kalo semua bisa di tindak harus jelas dasa...
Name: comment, Length: 7554, dtype: object

## Cleaning mark quotion etc.

In [10]:
import re
import string

def clean_mark(data):
    # Menghapus mention (@username) yang terdiri dari satu kata (hanya huruf dan angka)
    data = re.sub(r'@[A-Za-z0-9]+(?:_[A-Za-z0-9]+)*', '', data)
    
    # Menghapus hashtag (#hashtag)
    data = re.sub(r'#[A-Za-z0-9]+', '', data)
    
    # Menghapus URL
    data = re.sub(r'http\S+', '', data)
    
    # Menghapus angka
    data = re.sub(r'[0-9]+', '', data)

    # Menghapus RT (retweet)
    data = re.sub(r'RT\s+', '', data)
    
    # Menghapus karakter non-huruf dan non-angka
    data = re.sub(r'[^A-Za-z0-9]', ' ', data)

    # Menghapus karakter yang berulang lebih dari dua kali
    data = re.sub(r'(.)\1+', r'\1\1', data)
    
    # Menghapus duplikasi kata berturut-turut
    data = re.sub(r'\b(\w+)( \1\b)+', r'\1', data)
    
    # Menghapus tanda baca berturut-turut
    data = re.sub(r'[\?\.\!]+(?=[\?.\!])', '', data)
    
    # Menghapus tanda baca
    data = data.translate(str.maketrans('', '', string.punctuation))
    
    # Menghapus spasi di awal dan akhir
    data = data.strip()
    
    return data


def remove_emojis(text):
    emoji_pattern = re.compile(
        u'['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbol
        u'\U0001F680-\U0001F6FF'  # transports and map
        u'\U0001F700-\U0001F77F'  # alchemist symbols
        u'\U0001F780-\U0001F7FF'  # geometry symbols
        u'\U0001F800-\U0001F8FF'  # additional symbols
        u'\U0001F900-\U0001F9FF'  # faces, hands, etc.
        u'\U0001FA00-\U0001FA6F'  # additional faces
        u'\U0001FA70-\U0001FAFF'  # additional symbols
        u'\U00002702-\U000027B0'  # hand gestures, symbols
        u'\U000024C2-\U0001F251'  # general emojis
        u'\U0001F004-\U0001F0CF'  # playing card symbols
        u'\U0001F000-\U0001F02F'  # Mahjong
        u'\U0001F1E0-\U0001F1FF'  # Flags
        u'\U0001F200-\U0001F251'  # Enclosed characters
        u']', re.UNICODE)
    
    return re.sub(emoji_pattern, '', text)


In [11]:
sentimen_public['comment'] = sentimen_public['comment'].apply(clean_mark)
sentimen_public['comment'] = sentimen_public['comment'].apply(remove_emojis)

sentimen_public

C:\Users\alber\AppData\Local\Temp\ipykernel_6952\2557478904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentimen_public['comment'] = sentimen_public['comment'].apply(clean_mark)
C:\Users\alber\AppData\Local\Temp\ipykernel_6952\2557478904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentimen_public['comment'] = sentimen_public['comment'].apply(remove_emojis)


,username,comment
0,@alvinvinoza,naikin gajinya
1,@MuklasinMuklasin-k6w,kalo tidak ada hukuman mati sampek kiamat pun ...
2,@infotodax,gua gak kebayang gaji elu berapa tapi keseraka...
3,@febbriyansaputra4132,tolong host nya di support dan di lindungi ini
4,@chalvit4162,harusnya berita ya kayak begini dibawa santai ...
...,...,...
7549,hedianti,halah selama korupsi ga hukum mati cmn beberap...
7550,Charles39953441,beberapa dirut pertamina jadi tersangka setela...
7551,Aryabarcelona,tahun baru terungkap beneran gak si kerjanya
7552,harley_doni,masak ka sebodoh itu ya apakah mmg membodohkan...


In [13]:
sentimen_public['comment'].isna().sum()

0

In [14]:
sentimen_public.to_csv('sentiment_public.csv', index=False)